In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

data = pd.read_csv("meteo_data.csv")
data = data.drop(data.index[:12995])
data
data.to_csv('week.csv')

In [17]:
data = pd.read_csv("week.csv")
data = data.drop(data.columns[0], axis=1)
data

,SOLAR_RADIATION,PRECIPITATION,WIND_SPEED,LEAF_WETNESS,HC_AIR_TEMPERATURE,HC_RELATIVE_HUMIDITY,DEW_POINT
0,452.0,0.0,0.9,0.0,13.15,0.00,0.0
1,315.0,0.0,0.4,0.0,13.64,0.00,0.0
2,157.0,0.0,0.3,0.0,13.24,0.00,0.0
3,33.0,0.0,0.0,0.0,12.05,0.00,0.0
4,0.0,0.0,0.0,0.0,9.46,0.00,0.0
...,...,...,...,...,...,...,...
163,115.0,0.0,0.4,0.0,2.13,0.00,0.0
164,287.0,0.0,1.7,0.0,2.95,0.00,0.0
165,407.0,0.0,2.0,0.0,4.15,0.00,0.0
166,195.0,0.0,1.3,0.0,3.84,0.00,0.0


In [24]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Преобразование данных в массив numpy
dataset = data.values

# Масштабирование данных (нормализация)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

In [35]:
# Подготовка данных для LSTM модели
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data)-seq_length):
        X.append(data[i:i+seq_length, :])
        y.append(data[i+seq_length, 0])  # Возьмем первый столбец (SOLAR_RADIATION) в качестве целевой переменной
    return np.array(X), np.array(y)

sequence_length = 24  # Длина последовательности, например, на 24 часа
X, y = create_sequences(scaled_data, sequence_length)

# Разделение данных на обучающий и тестовый наборы
train_size = int(len(X) * 0.8)  # 70% данных используется для обучения
test_size = len(X) - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [36]:
# Создание и компиляция LSTM модели
model = Sequential([
    LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(units=1)
])
model.compile(optimizer='adam', loss='mse')

# Обучение модели
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Оценка модели на тестовом наборе данных
mse = model.evaluate(X_test, y_test)
print('Mean Squared Error on Test Set:', mse)

# Предсказание на тестовом наборе данных
y_pred = model.predict(X_test)

# Вычисление коэффициента детерминации
r2 = r2_score(y_test, y_pred)
print('R^2 Score on Test Set:', r2)

Epoch 1/50


C:\Users\Счастливый Наиль\PycharmProjects\meteoApp\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - loss: 0.0967 - val_loss: 0.0352
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0761 - val_loss: 0.0495
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0682 - val_loss: 0.0633
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0783 - val_loss: 0.0641
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0643 - val_loss: 0.0523
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0553 - val_loss: 0.0405
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0787 - val_loss: 0.0415
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0669 - val_loss: 0.0528
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0605 - val_loss: 0.0639
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0613 - val_loss: 0.0636
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0557 - val_loss: 0.0583
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0515 - val_loss: 0.0416
Epoch 13/50